In [2]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import time
import os
import cv2
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


def read_kaggle_data():
    imageList = []
    angList = []
    with open("/kaggle/input/car-steering-angle-prediction/driving_dataset/angles.txt") as ang:
        for x in ang:
            angList.append(float(x.split()[1]))
            f = x.split()[0]
            image = cv2.imread('/kaggle/input/car-steering-angle-prediction/driving_dataset/' + f)
            image_u = cv2.resize(image, (200,66), interpolation = cv2.INTER_AREA) #in given paper
            imageList.append(torch.from_numpy(image_u.transpose()).float())
    return list(zip(imageList,angList))


dataset = read_kaggle_data()

train_x = dataset[0: (len(dataset) - 13000)]
val_x = dataset[(len(dataset) - 13000) : (len(dataset) - 10000)]
test_x = dataset[(len(dataset) - 10000) : (len(dataset))]


In [10]:
import torch.nn.functional as F

class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.normal = nn.LayerNorm([200,66])
        self.conv1 = nn.Conv2d(3,  24, stride=2, kernel_size=5 ,padding=0) 
        self.conv2 = nn.Conv2d(24, 36, stride=2, kernel_size=5, padding=0)
        self.conv3 = nn.Conv2d(36, 48, stride=2, kernel_size=5, padding=0)
        self.conv4 = nn.Conv2d(48, 64, stride=1, kernel_size=3, padding=0)
        self.conv5 = nn.Conv2d(64, 64, stride=1, kernel_size=3, padding=0)
        self.lin1 = nn.Linear(1152, 100)
        self.lin2 = nn.Linear(100, 50)
        self.lin3 = nn.Linear(50, 10)
        self.lin4 = nn.Linear(10, 1)
        
    def forward(self, x):
        x = self.normal(x)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        
        x = x.view(x.shape[0], -1)
        
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = F.relu(self.lin3(x))
        x = self.lin4(x)
        return x


net = ConvNet()
n_iter = 25
b_size = 250
mu = 1e-4
tr_loss_list, vl_loss_list = [], []
optm = torch.optim.Adam(net.parameters(), lr = mu)
rms = torch.nn.MSELoss(reduction = 'sum')

p_start = time.time()

for epoch in range(n_iter):
    lo = 0
    s_time = time.time()
    d_load = torch.utils.data.DataLoader(train_x, batch_size = b_size, shuffle = True)
    outp = "epoch " + str(epoch) + " | "
    for i, val in enumerate(d_load):
        optm.zero_grad()
        y_val = torch.reshape(val[1], (val[1].shape[0], 1)).type(torch.float32)
        loss = rms(net(val[0]), y_val)
        lo = lo + loss.item()
        loss.backward()
        optm.step()
    outp = outp + str("train RMSE : " + str(np.sqrt(lo / len(train_x)))) + " | "
    tr_loss_list.append(np.sqrt(lo / len(train_x)))

    v_load = torch.utils.data.DataLoader(val_x, batch_size = b_size, shuffle = True)
    lo_tmp = 0

    for i, val in enumerate(v_load):
        y_val = torch.reshape(val[1], (val[1].shape[0], 1)).type(torch.float32)
        loss = rms(net(val[0]), y_val)
        lo_tmp = lo_tmp + loss.item()
    outp = outp + "val RMSE : " + str(np.sqrt(lo_tmp / len(val_x))) + " | "

    e_time = time.time()
    outp = outp + 'Epoch time(s): ' + str(e_time - s_time) + " | "
    vl_loss_list.append(np.sqrt(lo_tmp / len(val_x)))

    print(outp)
    
t_load = torch.utils.data.DataLoader(test_x, batch_size = b_size, shuffle = True)
lo_test = 0

for i, tval in enumerate(t_load):
    y_val = torch.reshape(tval[1], (tval[1].shape[0], 1)).type(torch.float32)
    loss = rms(net(tval[0]), y_val)
    lo_test = lo_test + loss.item()

print("Testing RMSE : ", vl_loss_list)

p_end = time.time()
print('Total execution time: ', (p_end - p_start))